In [1]:
## import libs
## FluidSynth install look for better options in future

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pylab as plt
from glob import glob
from PIL import Image
import librosa
import librosa.display
import IPython.display as ipd
import fluidsynth
from midi2audio import FluidSynth
from itertools import cycle
import os
from sklearn.preprocessing import minmax_scale

In [2]:
### extract features
def extract_features(sig, sr, fs, hs, composer):
    zcr = librosa.feature.zero_crossing_rate(y=sig, frame_length=fs, hop_length=hs)
    sc = librosa.feature.spectral_centroid(y=sig, sr=sr, n_fft=fs,
                                                          hop_length=hs)
    sb = librosa.feature.spectral_bandwidth(y=sig, sr=sr, n_fft=fs,
                                                            hop_length=hs)
    sroff = librosa.feature.spectral_rolloff(y=sig, sr=sr, n_fft=fs, hop_length=hs)
    mfcc = librosa.feature.mfcc(y=sig, sr=sr, n_fft=fs, hop_length=hs)
    scont = librosa.feature.spectral_contrast(y=sig, sr=sr, n_fft=fs,
                                                          hop_length=hs)
    
    return [

        np.mean(zcr),#zcr
        np.std(zcr), 
        np.mean(sc), #spectral_centroid
        np.std(sc),
        np.mean(sb), #spectral_bandwidth
        np.std(sb),#spectral_bandwidth
        np.mean(sroff),#spectral_rolloff
        np.std(sroff), # std
        np.mean(scont),#spectral_contrast
        np.std(scont),
        np.mean(mfcc[1, :]),
        np.std(mfcc[1, :]),
        np.mean(mfcc[2, :]),
        np.std(mfcc[2, :]),
        np.mean(mfcc[3, :]),
        np.std(mfcc[3, :]),
        np.mean(mfcc[4, :]),
        np.std(mfcc[4, :]),
        np.mean(mfcc[5, :]),
        np.std(mfcc[5, :]),
        np.mean(mfcc[6, :]),
        np.std(mfcc[6, :]),
        np.mean(mfcc[7, :]),
        np.std(mfcc[7, :]),
        np.mean(mfcc[8, :]),
        np.std(mfcc[8, :]),
        np.mean(mfcc[9, :]),
        np.std(mfcc[9, :]),
        np.mean(mfcc[10, :]),
        np.std(mfcc[10, :]),
        np.mean(mfcc[11, :]),
        np.std(mfcc[11, :]),
        np.mean(mfcc[12, :]),
        np.std(mfcc[12, :]),
        np.mean(mfcc[13, :]),
        np.std(mfcc[13, :]),
        composer
    ]

In [3]:
## loading to librosa
def load_file(file):
    y,sr = librosa.load(file)
    return y,sr

In [4]:
## helper to get all composer files
def get_all_composer_files(path):
    audio_files = glob(path)
    return audio_files

In [5]:
### get all files per composer
audio_files_bach = get_all_composer_files('../deloitte/PartOne/PS1/Bach/split_files/*')
audio_files_beet = get_all_composer_files('../deloitte/PartOne/PS1/Beethoven/split_files/*')
audio_files_brahms = get_all_composer_files('../deloitte/PartOne/PS1/Brahms/split_files/*')
audio_files_schubert = get_all_composer_files('../deloitte/PartOne/PS1/Schubert/split_files/*')
print(len(audio_files_bach))
print(len(audio_files_beet))
print(len(audio_files_brahms))
print(len(audio_files_schubert))


126
1692
326
374


In [6]:
### features
columns_frame = ['zcr_mean', 
         'zcr_std',
        'sc_mean',
         'sc_std',
         'scont_mean',
         'scont_std',
         'sb_mean',
         'sb_std',
         'sroff_mean',
         'sroff_std',
         'mfccs1_mean',
         'mfccs1_std',
         'mfccs2_mean',
         'mfccs2_std',
         'mfccs3_mean',
         'mfccs3_std',
         'mfccs4_mean',
         'mfccs4_std',
         'mfccs5_mean',
         'mfccs5_std',
         'mfccs6_mean',
         'mfccs6_std',
         'mfccs7_mean',
         'mfccs7_std',
         'mfccs8_mean',
         'mfccs8_std',
         'mfccs9_mean',
         'mfccs9_std',
         'mfccs10_mean',
         'mfccs10_std',
         'mfccs11_mean',
         'mfccs11_std',
         'mfccs12_mean',
         'mfccs12_std',
         'mfccs13_mean',
         'mfccs13_std',
         'composer'
    ]

In [31]:
### extract features from files
### also add a return column containing composer name
### for PS2 target, assign column with file name
def extract_features_from_files(audio_files,fs,hs,composer):
    res = []
    i=0
    #### to handle targets PS2 along with files with composers
    if composer==None:
        check_composer = False
    else:
        check_composer = True
        
    for file in audio_files:
        if check_composer is False:
            composer= os.path.basename(file)
        i+=1
        FRAME_SIZE = fs 
        HOP_SIZE = hs
        y, sr = load_file(file)
        res.append(extract_features(y, sr,FRAME_SIZE, HOP_SIZE,composer))
    print(f'processed {i}')
    return pd.DataFrame(res, columns = columns_frame)

In [10]:
df_bach = extract_features_from_files(audio_files_bach,2048,512,"Bach")
df_beet = extract_features_from_files(audio_files_beet,2048,512,"Beethoven")
df_brahms = extract_features_from_files(audio_files_brahms,2048,512,"Brahms")
df_shub = extract_features_from_files(audio_files_schubert,2048,512, "Schubert")


processed 326
processed 374


In [12]:
print(df_bach.shape)
print(df_beet.shape)
print(df_brahms.shape)
print(df_schub.shape)

(126, 37)
(1692, 37)
(326, 37)
(374, 37)


In [13]:
df_final = df_bach.append(df_beet).append(df_brahms).append(df_schub)
df_final.to_csv("feats.csv", index=False)

/var/folders/08/dpzlrd_n27zb6zw32lf533640000gn/T/ipykernel_7108/274115825.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_final = df_bach.append(df_beet).append(df_brahms).append(df_shub)
/var/folders/08/dpzlrd_n27zb6zw32lf533640000gn/T/ipykernel_7108/274115825.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_final = df_bach.append(df_beet).append(df_brahms).append(df_shub)


In [15]:
audio_files_target = glob('../deloitte/PartOne/PS2/split_files30/*')
df_target = extract_features_from_files(audio_files_target,2048,512,None)
df_target.to_csv("target_feats.csv",index=False)

processed 347


# lets do 15sec

In [17]:
### get all files per composer
audio_files_bach = get_all_composer_files('../deloitte/PartOne/PS1/Bach/split_files15/*')
audio_files_beet = get_all_composer_files('../deloitte/PartOne/PS1/Beethoven/split_files15/*')
audio_files_brahms = get_all_composer_files('../deloitte/PartOne/PS1/Brahms/split_files15/*')
audio_files_schubert = get_all_composer_files('../deloitte/PartOne/PS1/Schubert/split_files15/*')
print(len(audio_files_bach))
print(len(audio_files_beet))
print(len(audio_files_brahms))
print(len(audio_files_schubert))

242
3308
640
737


In [18]:
df_bach = extract_features_from_files(audio_files_bach,1024,256,"Bach")
df_beet = extract_features_from_files(audio_files_beet,1024,256,"Beethoven")
df_brahms = extract_features_from_files(audio_files_brahms,1024,256,"Brahms")
df_schub = extract_features_from_files(audio_files_schubert,1024,256, "Schubert")


processed 242


/Users/saurav/opt/anaconda3/envs/audio/lib/python3.9/site-packages/librosa/core/spectrum.py:257: UserWarning: n_fft=1024 is too large for input signal of length=684
  warnings.warn(
/Users/saurav/opt/anaconda3/envs/audio/lib/python3.9/site-packages/librosa/core/spectrum.py:257: UserWarning: n_fft=1024 is too large for input signal of length=662
  warnings.warn(
/Users/saurav/opt/anaconda3/envs/audio/lib/python3.9/site-packages/librosa/core/spectrum.py:257: UserWarning: n_fft=1024 is too large for input signal of length=22
  warnings.warn(


processed 3308
processed 640
processed 737


In [19]:
print(df_bach.shape)
print(df_beet.shape)
print(df_brahms.shape)
print(df_schub.shape)

(242, 37)
(3308, 37)
(640, 37)
(737, 37)


In [20]:
df_final = df_bach.append(df_beet).append(df_brahms).append(df_schub)
df_final.to_csv("feats15.csv", index=False)

/var/folders/08/dpzlrd_n27zb6zw32lf533640000gn/T/ipykernel_7108/911208544.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_final = df_bach.append(df_beet).append(df_brahms).append(df_schub)
/var/folders/08/dpzlrd_n27zb6zw32lf533640000gn/T/ipykernel_7108/911208544.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_final = df_bach.append(df_beet).append(df_brahms).append(df_schub)


In [32]:
audio_files_target = glob('../deloitte/PartOne/PS2/split_files15/*')
df_target = extract_features_from_files(audio_files_target,1024,256,None)
df_target.to_csv("target_feats15.csv",index=False)

processed 671


# lets do 1 min

In [23]:
### get all files per composer
audio_files_bach = get_all_composer_files('../deloitte/PartOne/PS1/Bach/split_files60/*')
audio_files_beet = get_all_composer_files('../deloitte/PartOne/PS1/Beethoven/split_files60/*')
audio_files_brahms = get_all_composer_files('../deloitte/PartOne/PS1/Brahms/split_files60/*')
audio_files_schubert = get_all_composer_files('../deloitte/PartOne/PS1/Schubert/split_files60/*')
print(len(audio_files_bach))
print(len(audio_files_beet))
print(len(audio_files_brahms))
print(len(audio_files_schubert))

67
883
166
193


In [24]:
df_bach = extract_features_from_files(audio_files_bach,4096,1024,"Bach")
df_beet = extract_features_from_files(audio_files_beet,4096,1024,"Beethoven")
df_brahms = extract_features_from_files(audio_files_brahms,4096,1024,"Brahms")
df_schub = extract_features_from_files(audio_files_schubert,4096,1024, "Schubert")


processed 67


/Users/saurav/opt/anaconda3/envs/audio/lib/python3.9/site-packages/librosa/core/spectrum.py:257: UserWarning: n_fft=4096 is too large for input signal of length=22
  warnings.warn(


processed 883
processed 166
processed 193


In [25]:
print(df_bach.shape)
print(df_beet.shape)
print(df_brahms.shape)
print(df_schub.shape)

(67, 37)
(883, 37)
(166, 37)
(193, 37)


In [26]:
df_final = df_bach.append(df_beet).append(df_brahms).append(df_schub)
df_final.to_csv("feats60.csv", index=False)

/var/folders/08/dpzlrd_n27zb6zw32lf533640000gn/T/ipykernel_7108/713426919.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_final = df_bach.append(df_beet).append(df_brahms).append(df_schub)
/var/folders/08/dpzlrd_n27zb6zw32lf533640000gn/T/ipykernel_7108/713426919.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_final = df_bach.append(df_beet).append(df_brahms).append(df_schub)


In [33]:
audio_files_target = glob('../deloitte/PartOne/PS2/split_files60/*')
df_target = extract_features_from_files(audio_files_target,4096,1024,None)
df_target.to_csv("target_feats60.csv",index=False)

processed 183
